In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb
from catboost import CatBoostClassifier

In [2]:
columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv('car.data', names=columns)
car_data.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [3]:
car_encoded = pd.get_dummies(car_data, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

X = car_encoded.drop('class', axis=1)
y = car_encoded['class']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)



In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [8]:
#TODO debug xgboost import
xgb_data= xgb.XGBClassifier(objective='multi:softmax', num_class=3)
xgb_data.fit(X_train, y_train)
y_pred_xgb = xgb_data.predict(X_test)

print("XGBoost")
print(classification_report(y_test, y_pred_xgb))

XGBoost
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        83
           1       0.61      1.00      0.76        11
           2       1.00      1.00      1.00       235
           3       1.00      1.00      1.00        17

    accuracy                           0.98       346
   macro avg       0.90      0.98      0.93       346
weighted avg       0.99      0.98      0.98       346



In [11]:
catboost_data = CatBoostClassifier(iterations=150, depth=5, learning_rate=0.3, loss_function='MultiClass', verbose=15)
catboost_data.fit(X_train, y_train)
y_pred_catboost = catboost_data.predict(X_test, prediction_type='Class')

print("\nCatboost")
print(classification_report(y_test, y_pred_catboost))

0:	learn: 0.9474975	total: 1.36ms	remaining: 202ms
15:	learn: 0.2026547	total: 25.6ms	remaining: 214ms
30:	learn: 0.1136741	total: 50.3ms	remaining: 193ms
45:	learn: 0.0760837	total: 76.7ms	remaining: 173ms
60:	learn: 0.0548474	total: 102ms	remaining: 149ms
75:	learn: 0.0425822	total: 128ms	remaining: 124ms
90:	learn: 0.0346446	total: 153ms	remaining: 98.9ms
105:	learn: 0.0287692	total: 178ms	remaining: 73.7ms
120:	learn: 0.0247338	total: 203ms	remaining: 48.7ms
135:	learn: 0.0219709	total: 228ms	remaining: 23.5ms
149:	learn: 0.0195640	total: 251ms	remaining: 0us

Catboost
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        83
           1       0.59      0.91      0.71        11
           2       1.00      1.00      1.00       235
           3       0.94      1.00      0.97        17

    accuracy                           0.98       346
   macro avg       0.88      0.96      0.91       346
weighted avg       0.98      0.98      0